# Imports

In [16]:
import copy
import random
from datetime import datetime, timedelta, timezone
from importlib.resources import files
# from shodan.shodan.shodan import Shodan
from pprint import pprint, pformat
from time import time

from ledhntr import LEDHNTR
from ledhntr.plugins import HNTRPlugin
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)
from ledhntr.helpers import dumps

led = LEDHNTR()
# shodan: Shodan
shodan = led.plugins['shodan']
censys = led.plugins['censys']
zeta = led.plugins['zeta']


def simple_search(
    plugin: HNTRPlugin = None,
    query: str = "",
    endpoint: str = "",
)->dict:

    plugin_name = plugin.__class__.__name__.lower()
    now = datetime.now(timezone.utc)
    # hosts_search

    res = plugin.search(simple={'endpoint': endpoint, 'query':query})
    data = dumps(res['raw'], compactly=True)
    date = f"{now.year}{now.month}{now.day}T{now.hour}{now.minute}{now.second}"
    with open(
        f'./data/{plugin_name}-{date}-{endpoint}.json', 
        'w'
    ) as f:
        f.write(data)
    return res

2024-05-23 09:25:48 [INFO] ledhntr[19744] > core.py > (__init__) [94] >  Successfully loaded configs!


# Shodan

In [8]:
now = datetime.now(timezone.utc)
plugin = shodan
plugin_name = plugin.__class__.__name__.lower()

# hosts_search
q = '''hash:-1587139099 ssl:"CN=ocserv VPN, O=ocserv"'''
endpoint = 'hosts_search'
res = simple_search(plugin, q, endpoint)

# host_details
query_number = 1
ips = []

for thing in res['things']:
    if thing.label=='ip':
        ips.append(thing.keyval)
        if len(ips) >= query_number:
            break

endpoint = 'host_details'
for ip in ips:
    res = simple_search(plugin, ip, endpoint)



2024-05-23 08:23:33 [INFO] ledhntr[19744] > hntr.py > (search) [2270] >  Running Shodan search against endpoint hosts_search
2024-05-23 08:23:35 [INFO] ledhntr[19744] > hntr.py > (_inc_api_counter) [940] >  Total Shodan API calls:
{'_total': 13, 'host_details': 11, 'hosts_search': 2}
2024-05-23 08:23:35 [INFO] ledhntr[19744] > shodan.py > (new_parse_hosts_search) [302] >  Running new parsers for /shodan/host/search/ results...
2024-05-23 08:23:54 [INFO] ledhntr[19744] > hntr.py > (search) [2270] >  Running Shodan search against endpoint host_details
2024-05-23 08:23:55 [INFO] ledhntr[19744] > hntr.py > (_inc_api_counter) [940] >  Total Shodan API calls:
{'_total': 14, 'host_details': 12, 'hosts_search': 2}


# Censys

In [11]:
now = datetime.now(timezone.utc)
plugin = censys
plugin_name = plugin.__class__.__name__.lower()

# hosts_search
q = '''same_service(
    services.tls.certificates.leaf_data.subject_dn="CN=ocserv VPN, O=ocserv" 
    AND services.banner_hashes="sha256:2206fcd3ea22d4dcf52f4dd9291753b80e0533e93586597e3c78b417a1537290"
)'''
endpoint = 'search'
res1 = simple_search(plugin, q, endpoint)

# host_details
query_number = 1
ips = []

for thing in res1['things']:
    if thing.label=='ip':
        ips.append(thing.keyval)
        if len(ips) >= query_number:
            break

endpoint = 'host_details'
for ip in ips:
    res2 = simple_search(plugin, ip, endpoint)



2024-05-23 08:32:43 [INFO] ledhntr[19744] > hntr.py > (search) [2270] >  Running Censys search against endpoint search


2024-05-23 08:32:44 [INFO] ledhntr[19744] > hntr.py > (_inc_api_counter) [940] >  Total Censys API calls:
{'_total': 3, 'host_details': 1, 'search': 2}
2024-05-23 08:32:44 [INFO] ledhntr[19744] > censys.py > (parse_hosts_search) [276] >  Parsing /hosts/search/ results..
2024-05-23 08:32:44 [INFO] ledhntr[19744] > censys.py > (parse_hosts_search) [409] >  Parsed 178 things from 98 IPs!
2024-05-23 08:32:44 [INFO] ledhntr[19744] > hntr.py > (search) [2270] >  Running Censys search against endpoint host_details
2024-05-23 08:32:44 [INFO] ledhntr[19744] > hntr.py > (rate_limit_respect) [1897] >  Rate limit not hit! Waiting 0.9913866519927979 seconds...
2024-05-23 08:32:46 [INFO] ledhntr[19744] > hntr.py > (_inc_api_counter) [940] >  Total Censys API calls:
{'_total': 4, 'host_details': 2, 'search': 2}
2024-05-23 08:32:46 [INFO] ledhntr[19744] > censys.py > (parse_hosts) [502] >  Parsing hosts...
2024-05-23 08:32:46 [INFO] ledhntr[19744] > censys.py > (parse_hosts) [1088] >  Found duplicate 

# Zeta

In [15]:
now = datetime.now(timezone.utc)
plugin = zeta
plugin_name = plugin.__class__.__name__.lower()
domain = "poetpages.com"
ip = "65.21.51.58"

# hostname
endpoint = 'hostname'
zreshostname = simple_search(plugin, domain, endpoint)

# ip
endpoint = "ip"
zresip = simple_search(plugin, ip, endpoint)

# domain2d8s
endpoint = "domain2d8s"
zresd8s = simple_search(plugin, domain, endpoint)

# livedns
endpoint = "livedns"
zresdns = simple_search(plugin, domain, endpoint)

<Entity(label=ip,con=0.0,ip-address=152.70.83.47,has=4)
<Relation(label=geoloc,con=0.0,has=9,active_roles=1,total_players=1)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=38.180.95.46,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=103.42.214.218,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=119.28.6.142,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=8.140.194.126,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=103.51.140.101,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=81.181.106.124,has=4)
<Relation(label=cidr,con=0.0,has=3,active_roles=2,total_players=2)
<Entity(label=ip,con=0.0,ip-address=43.128.136.124,has=4)
<Rela